In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://apache.mirror.digitalpacific.com.au/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-08-15 10:19:08--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-08-15 10:19:08 (6.06 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema = True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [5]:
f"There are {df.count()} rows in dataset"

'There are 85981 rows in dataset'

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
from pyspark.sql.functions import to_date

In [8]:
# CREATE TABLE review_id_table
review_id_table = df.select(['review_id', 'customer_id','product_id', 'product_parent', to_date("review_date","yyyy-mm-dd").alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1QX6706ZWJ1P5|   32114233|B00OYRW4UE|     223980852| 2015-01-31|
|R3QWMLJHIW6P37|   18125776|B0000537JQ|     819771537| 2015-01-31|
|R14Z1VR1N0Z9G6|   19917519|B00HXXO332|     849307176| 2015-01-31|
| R25ZRJL0GH0U0|   18277171|B00EOB0JA2|     700864740| 2015-01-31|
|R3837KYH7AZNIY|    2593270|B00OC2O1UC|     794298839| 2015-01-31|
|R2MN0QYCY6EVIV|    2592955|B00HES9CMS|     318730927| 2015-01-31|
|R3AN2UJ1D42ZS0|   15168265|B0016BFR4G|     887476137| 2015-01-31|
|R3U29ZLUWEEK4M|   13761624|B00K504UUG|     458706868| 2015-01-31|
|R16ZDMJJHK796C|   37070734|B00HES9CMS|     318730927| 2015-01-31|
| RRRDOEJZD1Y22|   29615023|B00P6TUO5G|     170248843| 2015-01-31|
|R2KR5ZEAS859DK|   47893062|B0006VJ6TO|     412568457| 2015-01-31|
| RR7PGQY763IHF|    2582596|B00H9L7VIW|     851045898| 2015-01

In [9]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
# CREATE TABLE products (product_id + product_title)
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00J7G8EL0|GUM Expanding Flo...|
|B000BGI4DS|MedWeigh MS-2510 ...|
|B00CADD4R6|Cigarette Injecto...|
|B008LEPP3O|Cytomax Powder-Or...|
|B00NTRVVA4|Beauty Wig World ...|
|B000KE8ASK|Inc Magnetic Peda...|
|B00U11YXD8|SG Rubber Foam Mu...|
|B008Y7EZMU|"ABC Products" - ...|
|B00075VVYC|Reizen RL-350 Bra...|
|B005P2KHB0|AUTHENTIC - DESIG...|
|B0002UBEGW|SASSOON HAIR TRIM...|
|B00BHEO7RK|Recliner Risers S...|
|B00GIOTGPW|Hot Tools Dryer -...|
|B00GIX9GLW|Bundle - Doc John...|
|B00GOTNA3A|Wet Light Liquid ...|
|B003VJETUO|BooJee 6 Pk Booje...|
|B003BIFRFQ|Pjur Backdoor Rel...|
|B00HHP2QQ2|ULTRA DNA TM 50 S...|
|B001QGMXAW|Drive Medical Car...|
|B0021YY0YK|Medline ORT27200 ...|
+----------+--------------------+
only showing top 20 rows



In [11]:
# CREATE TABLE customers (customer_id + customer_count)
customer_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11160821|             1|
|    1099354|             1|
|   37200466|             1|
|   31581262|             1|
|   27596904|             1|
|    2405598|             1|
|   11229098|             1|
|   17237226|             1|
|   46000147|             1|
|   15234640|             1|
|   28937218|             1|
|   43920023|             1|
|   27954895|             1|
|    8269764|             1|
|   31689770|             1|
|   30117841|             1|
|   38591899|             1|
|   24557051|             1|
|    2780908|             1|
|   32466409|             1|
+-----------+--------------+
only showing top 20 rows



In [12]:
# CREATE TABLE vine_table (review_id TEXT PRIMARY KEY,star_rating INTEGER,helpful_votes INTEGER,total_votes INTEGER,vine TEXT
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1QX6706ZWJ1P5|          5|            0|          0|   N|
|R3QWMLJHIW6P37|          5|            0|          0|   N|
|R14Z1VR1N0Z9G6|          5|            1|          1|   N|
| R25ZRJL0GH0U0|          2|            0|          0|   N|
|R3837KYH7AZNIY|          4|            0|          1|   N|
|R2MN0QYCY6EVIV|          5|            0|          0|   N|
|R3AN2UJ1D42ZS0|          5|            0|          0|   N|
|R3U29ZLUWEEK4M|          5|            0|          0|   N|
|R16ZDMJJHK796C|          5|            0|          0|   N|
| RRRDOEJZD1Y22|          4|            0|          0|   N|
|R2KR5ZEAS859DK|          5|            0|          0|   N|
| RR7PGQY763IHF|          5|            0|          0|   N|
| RN37YYZBP9BMP|          5|            4|          4|   N|
| R1UXGB70OR5TW|          5|            

In [13]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cbal3aw7dlkn.us-east-2.rds.amazonaws.com:5432/amazon_reviews_db"
config = {"user":"root", 
          "password": "bigdatachallenge", 
          "driver":"org.postgresql.Driver"}

In [ ]:
 # Write DataFrame to the review_id table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to the products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to the customers table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to the vine table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)